In [31]:
from utilities.proj1_helpers import load_csv_data

In [32]:
train_data = load_csv_data("data/train.csv")
y, x, ids = train_data[0], train_data[1], train_data[2]
print(y.shape, x.shape, ids.shape)

(250000,) (250000, 30) (250000,)


In [33]:
test_data = load_csv_data("data/test.csv")
y_train, x_train, ids_train = train_data[0], train_data[1], train_data[2]

In [34]:
from regression.linear_models import *
from regression.stochastic_models import *
from utilities.cross_validation import cross_validation, build_k_indices, split_data
from regression.loss import compute_rmse_loss
from utilities.pca import compute_pca
from utilities.stochastic import *

In [35]:
# Params to bruteforce
lambdas = np.logspace(-5, 1, 10)
degrees = np.arange(1, 8, 1)
k_folds = 4
seed = 1

In [36]:
x_reduced = compute_pca(x)
x_train_reduced = compute_pca(x_train)
indices = np.random.randint(0, high=len(x), size=((1, 500)))

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter3D(x_reduced[indices, 0], x_reduced[indices, 0], x_reduced[indices, 2])
plt.show()

In [13]:
def generate_w(num_intervals):
    """Generate a grid of values for w0 and w1."""
    w0 = np.linspace(-100, 200, num_intervals)
    w1 = np.linspace(-100, 200, num_intervals)
    w2 = np.linspace(-100, 200, num_intervals)
    return w0, w1, w2


def get_best_parameters(w0, w1, w2, losses):
    """Get the best w from the result of grid search."""
    min_row, min_col, min_slice = np.unravel_index(np.argmin(losses), losses.shape)
    return losses[min_row, min_col, min_slice], w0[min_row], w1[min_col], w2[min_slice]

def grid_search(y, tx, w0, w1, w2):
    """Algorithm for grid search."""
    losses = np.zeros((len(w0), len(w1), len(w2)))
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO: compute loss for each combination of w0, w1 and w2.
    # ***************************************************
    for i in range(0, len(w0)):
        for j in range(0, len(w1)):
            for k in range(0, len(w2)):
                losses[i, j, k] = compute_rmse_loss(y, tx, np.array([w0[i], w1[j], w2[k]]))
    return losses

w0, w1, w2 =  generate_w(30)
losses = grid_search(y, x_reduced, w0, w1, w2)
print(get_best_parameters(w0, w1, w2, losses))

(964.75015707555065, 3.448275862068968, -6.8965517241379359, 3.448275862068968)


In [37]:
x_reduced.shape

(250000, 3)

In [38]:
best_loss = 1e100
best_params = (-1, -1)
k_indices = build_k_indices(y, k_folds, seed)
degree = 1

D = x_reduced.shape[1]
w0 = np.array([0 for i in range(D)])

for l in lambdas:
    w, loss_tr = stochastic_gradient_descent(y, x_reduced, w0, 1000, 1e-10, compute_rmse_loss, compute_mse_gradient)
    loss_te = compute_rmse_loss(y_train, x_train_reduced, w, lambda_=l)
    if loss_te < best_loss:
        best_loss = loss_te
        best_params = (l, degree)

    print("l={}, d={}: train {}, test {}".format(l, degree, loss_tr, loss_te))

Stochastic Gradient Descent(0/999): loss=1.0, w0=1.0255085178016184e-09, w1=-5.304352866611016e-09
Stochastic Gradient Descent(1/999): loss=0.9999977876881669, w0=2.051003631731292e-09, w1=-1.0608499430157068e-08
Stochastic Gradient Descent(2/999): loss=0.9999955755782765, w0=3.076485342395999e-09, w1=-1.5912439700334164e-08
Stochastic Gradient Descent(3/999): loss=0.9999933636703106, w0=4.10195365040269e-09, w1=-2.1216173686838074e-08
Stochastic Gradient Descent(4/999): loss=0.999991151964251, w0=5.127408556358293e-09, w1=-2.6519701399363954e-08
Stochastic Gradient Descent(5/999): loss=0.9999889404600796, w0=6.152850060869696e-09, w1=-3.18230228476066e-08
Stochastic Gradient Descent(6/999): loss=0.9999867291577784, w0=7.178278164543767e-09, w1=-3.7126138041260294e-08
Stochastic Gradient Descent(7/999): loss=0.9999845180573291, w0=8.20369286798734e-09, w1=-4.242904699001889e-08
Stochastic Gradient Descent(8/999): loss=0.9999823071587141, w0=9.229094171807226e-09, w1=-4.773174970357576e

In [40]:
print(best_params, best_loss)

(1.0000000000000001e-05, 1) 0.997885609365


In [52]:
x_reduced

array([[ -64.89625544, -164.40788124, -701.09792732],
       [ -88.72849084,  162.88349515,  474.44739328],
       [ -24.68955588,  178.13773663,  441.01776584],
       ..., 
       [ -58.37144159,  178.51106485,  471.13413517],
       [ -56.30347342,  161.07629039,  512.35258349],
       [  18.60204858,  150.43432321,  500.47161825]])

In [54]:
def build_polynomial(x, degree):
    poly = x
    N, D = poly.shape
    for d in range(1, degree):
        poly_d = x[:, 0]*poly[:, ??]
        poly_d = np.hstack((poly_d, x[:, 1]*poly[:, D:]))
        poly_d = np.hstack((poly_d, x[:, 2]**(d+1)))
        poly = np.hstack((poly, poly_d))
        D = poly.shape[1]
    poly = np.hstack((np.ones((N, 1)), poly))
    return poly

build_polynomial(x_reduced, 2)

ValueError: operands could not be broadcast together with shapes (250000,) (250000,3) 